# If you're going to change something in this notebook after running it, restart the kernel - otherwise, it'll do it for you

In [ ]:
import os, sys, glob, pickle, tables
import numpy as np
# pip install simweights <--- run this once to install the package
# then replace <username> with your username and <version> with your Python version
sys.path.append('/home/<username>/.local/lib/python3.<version>/site-packages')
import simweights

In [3]:
p_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/p/20174_v1s/h5files/*.h5')
he_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/He/20178_v1s/h5files/*.h5')
o_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/O/20179_v1s/h5files/*.h5')
fe_list = glob.glob('/data/ana/CosmicRay/IceTop_level3/sim/IC86.2015/SIBYLL2.1/Fe/20180_v1s/h5files/*.h5')

bigfilelist = p_list + he_list + o_list + fe_list

In [ ]:
weighter = None
print('Getting weights...')
    
for filename in bigfilelist:
    file_obj = tables.open_file(filename, 'r')
    
    if weighter is None:
        weighter = simweights.IceTopWeighter(file_obj)
    else:
        weighter += simweights.IceTopWeighter(file_obj)

In [5]:
PICKLING_FILES = {
    #Energy, type
    'energy': weighter.get_column('MCPrimary', 'energy'),
    'particle_type': weighter.get_column('MCPrimary', 'type'),
    # Weights
    'Hweights': weighter.get_weights(simweights.GaisserH4a_IT()),
    'Gweights': weighter.get_weights(simweights.GlobalSplineFit_IT()),
    # Cuts
    'hits': weighter.get_column('IceTopHLCSeedRTPulses_SnowUnAttenuated_info', 'nstrings'),
    'max_inside': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'IceTopMaxSignalInside')==1,
    'runs': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'Run'),
    'event': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'Event'),
    'sub_event': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'SubEvent'),
    'sub_event_stream': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'SubEventStream'),
    'beta_cut_pass': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'BetaCutPassed'),
    'max_signal_6': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'IceTopMaxSignalAbove6'),
    'neighbour_max_signal_4': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'IceTopNeighbourMaxSignalAbove4'),
    'standard_filter': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'IceTop_StandardFilter'),
    'reco_pass': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'IceTop_reco_succeeded'),
    'laputop_frac': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'Laputop_FractionContainment'),
    'station_density_pass': weighter.get_column('IT73AnalysisIceTopQualityCuts', 'StationDensity_passed')
}

In [ ]:
for key, array in PICKLING_FILES.items():
    if not os.path.isfile(f'pickles/{key}.pkl'):
        print(f'Pickling {key}...')
        with open(f'pickles/{key}.pkl', 'wb') as f:
            pickle.dump(array, f)

print('Finished!')

: 